In [6]:
import pandas as pd
import numpy as np
import os 
from SSA import SSA_H_Plus
from FeatureExtractor import FeatureExtractor 
import scipy.io as sio
from statistics import mean 
from collections.abc import Iterable
import json
from math import sqrt 
from torch import nn
import torch
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import json

In [2]:
fe=FeatureExtractor() 
image_paths = []  
image_folder = os.getcwd()+'/data-set/CUB_200_2011/CUB_200_2011/images'

In [3]:
for path, subdirs, files in os.walk(image_folder):
    for name in files:
        if(name.endswith('.jpg')):
            image_paths.append(os.path.join(path, name)) 

In [4]:
print(len(image_paths))

11788


In [4]:
imgPath_feature=[]
imgPath_obj=[] 
feature_matrix=[]
k=0
for img_path in image_paths:
    feature=fe.extract(img_path)
    feature_matrix.append(feature)
    imgPath_obj.append(img_path)
    k=k+1
    print('VGG:'+str(k)) 
sio.savemat('cached-data/new/CUB/img_path_cub.mat',{"img_path_obj":imgPath_obj})

1/1 [==============================] - 0s 169ms/step


2022-11-30 23:42:56.791605: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


VGG:1
1/1 [==============================] - 0s 112ms/step
VGG:2
1/1 [==============================] - 0s 111ms/step
VGG:3
1/1 [==============================] - 0s 111ms/step
VGG:4
1/1 [==============================] - 0s 110ms/step
VGG:5
1/1 [==============================] - 0s 111ms/step
VGG:6
1/1 [==============================] - 0s 109ms/step
VGG:7
1/1 [==============================] - 0s 108ms/step
VGG:8
1/1 [==============================] - 0s 109ms/step
VGG:9
1/1 [==============================] - 0s 115ms/step
VGG:10
1/1 [==============================] - 0s 111ms/step
VGG:11
1/1 [==============================] - 0s 109ms/step
VGG:12
1/1 [==============================] - 0s 164ms/step
VGG:13
1/1 [==============================] - 0s 144ms/step
VGG:14
1/1 [==============================] - 0s 123ms/step
VGG:15
1/1 [==============================] - 0s 115ms/step
VGG:16
1/1 [==============================] - 0s 110ms/step
VGG:17
1/1 [==============================] - 0s 

In [5]:

k=0
ssa_plus_features=[]
for fet in feature_matrix:
    ssa_plus=SSA_H_Plus(fet)   
    ssa_plus_features.append(ssa_plus)
    k=k+1
    print('SSA:'+str(k))
sio.savemat('cached-data/new/CUB/ssa_features_cub.mat',{'ssa_features':ssa_plus_features})

SSA:1
SSA:2
SSA:3
SSA:4
SSA:5
SSA:6
SSA:7
SSA:8
SSA:9
SSA:10
SSA:11
SSA:12
SSA:13
SSA:14
SSA:15
SSA:16
SSA:17
SSA:18
SSA:19
SSA:20
SSA:21
SSA:22
SSA:23
SSA:24
SSA:25
SSA:26
SSA:27
SSA:28
SSA:29
SSA:30
SSA:31
SSA:32
SSA:33
SSA:34
SSA:35
SSA:36
SSA:37
SSA:38
SSA:39
SSA:40
SSA:41
SSA:42
SSA:43
SSA:44
SSA:45
SSA:46
SSA:47
SSA:48
SSA:49
SSA:50
SSA:51
SSA:52
SSA:53
SSA:54
SSA:55
SSA:56
SSA:57
SSA:58
SSA:59
SSA:60
SSA:61
SSA:62
SSA:63
SSA:64
SSA:65
SSA:66
SSA:67
SSA:68
SSA:69
SSA:70
SSA:71
SSA:72
SSA:73
SSA:74
SSA:75
SSA:76
SSA:77
SSA:78
SSA:79
SSA:80
SSA:81
SSA:82
SSA:83
SSA:84
SSA:85
SSA:86
SSA:87
SSA:88
SSA:89
SSA:90
SSA:91
SSA:92
SSA:93
SSA:94
SSA:95
SSA:96
SSA:97
SSA:98
SSA:99
SSA:100
SSA:101
SSA:102
SSA:103
SSA:104
SSA:105
SSA:106
SSA:107
SSA:108
SSA:109
SSA:110
SSA:111
SSA:112
SSA:113
SSA:114
SSA:115
SSA:116
SSA:117
SSA:118
SSA:119
SSA:120
SSA:121
SSA:122
SSA:123
SSA:124
SSA:125
SSA:126
SSA:127
SSA:128
SSA:129
SSA:130
SSA:131
SSA:132
SSA:133
SSA:134
SSA:135
SSA:136
SSA:137
SSA:138
SSA:

Randomize Data:

In [7]:
indices=list(range(int(len(imgPath_obj))))
np.random.shuffle(indices)
shuffle_image=[]
shuffle_ssa=[]

for index in indices:
     shuffle_image.append(imgPath_obj[index])
     shuffle_ssa.append(ssa_plus_features[index])
ssa_plus_features=shuffle_ssa
imgPath_obj=shuffle_image

Load Data:

In [20]:
ssa_plus_features=sio.loadmat('cached-data/new/CUB/ssa_features_cub.mat')['ssa_features']
imgPath_obj=sio.loadmat('cached-data/new/CUB/img_path_cub.mat')["img_path_obj"]

In [21]:
def GetNameAndClass(imgPath):
    splitted=str(imgPath).split('/')
    return {'Name':splitted[len(splitted)-1],'Class':splitted[len(splitted)-2]}

In [22]:
All_Master_Data=[]
for i in range(len(imgPath_obj)):
    NameAndClass=GetNameAndClass(imgPath_obj[i])
    All_Master_Data.append({
        'Name':NameAndClass['Name'], 'Class':NameAndClass['Class'], 'SSA':ssa_plus_features[i]
    })

In [26]:
train_data=All_Master_Data[20:]
test_data=All_Master_Data[:20]
print(len(train_data))
print(len(test_data))

11768
20


In [28]:
All_Results=[]
for test in test_data:
    Distances=[]
    for train in train_data:
        test_ssa=test['SSA']
        train_ssa=train['SSA']
        this_distance= sqrt( sum(np.multiply(test_ssa - train_ssa,test_ssa - train_ssa)))
        Distances.append([this_distance,train['Name'],train['Class']])
    Distances.sort()
    Distances=Distances[:5]
    All_Results.append({'Input':test['Class'],'Image':test['Name'],
    'Outputs':[
        Distances[0],
        Distances[1],
        Distances[2],
            Distances[3],
             Distances[4]
    ]
    })
    count_of_true=0
    In_Splitted=test['Name'].split('_')
 
with open('cached-data/new/cars/all_results.json', 'w') as f:
    json.dump([str(x) for x in All_Results], f)    

In [27]:
super_class_score=0
intra_class_score=0

for result in All_Results: 
    In_Splitted=result['Input'].split('_')
    SupperClassIn=In_Splitted[len(In_Splitted)-1]
    count_of_this_ture=0
    for out in result['Outputs']:
        out_splitted=out.split('_')
        SupperClassOut=out_splitted[len(out_splitted)-1]
        if(SupperClassIn==SupperClassOut):
            super_class_score=super_class_score+1
            count_of_this_ture=count_of_this_ture+1
        if(out==result['Input']):
            intra_class_score=intra_class_score+1 
count_of_results=len(All_Results)*5

print('Count of all results: '+str(count_of_results))
print('Count of super class score: '+str(super_class_score))
print('Count of intra class score: '+str(intra_class_score))

Count of all results: 100
Count of super class score: 85
Count of intra class score: 43


In [33]:
html='<table>'
for res in All_Results:
    tr='<tr>'
    td='<td>'
    InputClassSplitted=res['Input'].split('_')
    InputClass=InputClassSplitted[len(InputClassSplitted)-1]
    input='<img src="'+res['Input']+'/'+res['Image'].replace(' ','')+'"/>'+'<span>'+InputClass+'</span>'
    td=td+input+'</td>'
    tr=tr+td
    for out in res['Outputs']:
        tdout='<td>'
        OutoutClassSplitted=out[2].split('_')
        OutClass=OutoutClassSplitted[len(OutoutClassSplitted)-1]
        outimage='<img src="'+out[2]+'/'+out[1].replace(' ','')+'"/>'+'<span>'+OutClass+'</span>'
        tdout=tdout+outimage
        tr=tr+tdout
    tr=tr+'</tr>'
    html=html+tr
html=html+'</table>'
print(html)

<table><tr><td><img src="193.Bewick_Wren/Bewick_Wren_0082_185021.jpg"/><span>Wren</span></td><td><img src="181.Worm_eating_Warbler/Worm_Eating_Warbler_0055_795555.jpg"/><span>Warbler</span><td><img src="193.Bewick_Wren/Bewick_Wren_0075_185046.jpg"/><span>Wren</span><td><img src="069.Rufous_Hummingbird/Rufous_Hummingbird_0119_59681.jpg"/><span>Hummingbird</span><td><img src="197.Marsh_Wren/Marsh_Wren_0049_188540.jpg"/><span>Wren</span><td><img src="198.Rock_Wren/Rock_Wren_0059_188941.jpg"/><span>Wren</span></tr><tr><td><img src="189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0015_182320.jpg"/><span>Woodpecker</span></td><td><img src="189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0095_180948.jpg"/><span>Woodpecker</span><td><img src="189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0066_182253.jpg"/><span>Woodpecker</span><td><img src="189.Red_bellied_Woodpecker/Red_Bellied_Woodpecker_0054_182031.jpg"/><span>Woodpecker</span><td><img src="036.Northern_Flicker/Northern_Flicker_013